In [1]:
exec(open("./funcs/tool_funcs.py").read())


In [2]:
def dijkstra_show_path_minimax_path(adj_matrix, start, end):
 
    n = len(adj_matrix)
    distances = [float('inf')] * n
    previous_nodes = [-1] * n
    distances[start] = 0
    priority_queue = [(0, start)]  # (distance, node)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Stop if we reached the destination node
        if current_node == end:
            break

        # If the distance is no longer optimal, skip
        if current_distance > distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight > 0:  # Only consider edges with weight > 0
                distance = max(current_distance, weight)

                # Update distance if it's better
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

    # Reconstruct the path from end to start
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    # If the start node isn't reachable, return an empty path
    return path if path[0] == start else []

In [3]:
def translate_path(path, candidate_node_list):
    new_path = [candidate_node_list[i] for i in path]
    return new_path




In [4]:
def output_small_matrix(X_distance_matrix, candidate_node_list):
    small_matrix = np.zeros((len(candidate_node_list), len(candidate_node_list)))
    for i, ii in enumerate(candidate_node_list):
        for j, jj in enumerate(candidate_node_list):
            small_matrix[i,j] = X_distance_matrix[ii, jj]
    return small_matrix




In [5]:
def cal_candidate_node_list_minimax_path(i, j, minimax_path_matrix, X_distance_matrix):
    
    N = len(X_distance_matrix)
    
    remaining_list = [k for k in np.arange(N) if k != i and k != j]
    candidate_node_list = []
    candidate_node_list.append(i)
    for t in remaining_list:
        if minimax_path_matrix[i,j] < max(minimax_path_matrix[i,t], minimax_path_matrix[t,j]):
            pass
        else:
            candidate_node_list.append(t)
    candidate_node_list.append(j)
 
    return candidate_node_list
        



In [6]:

def cal_path_warm_start_minimax_path(i, j, minimax_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list_minimax_path(i, j, minimax_path_matrix, X_distance_matrix)
    # print(candidate_node_list)
   
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path_minimax_path(small_matrix, 0, len(candidate_node_list) - 1)
    path = translate_path(path, candidate_node_list)
    
    return path


In [7]:
# N = 100

# X_distance_matrix = create_distance_matrix(N)


In [8]:


X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")

minimax_path_matrix = floyd_warshall_minimax_path(X_distance_matrix)
 

In [9]:

N = len(X_distance_matrix)


In [10]:
i = 10
j = 50

path1 = dijkstra_show_path_minimax_path(X_distance_matrix, i, j)
    
path2 = cal_path_warm_start_minimax_path(i, j, minimax_path_matrix, X_distance_matrix)   
print(path1)
print(path2)


    

[10, 90, 20, 13, 95, 17, 9, 50]
[10, 90, 20, 13, 95, 17, 9, 50]


In [11]:
start = time.time()
for i in range(N):
    for j in range(N):
#         if i != j:
        if i == 2:
            path2 = cal_path_warm_start_minimax_path(i, j, minimax_path_matrix, X_distance_matrix)
end = time.time()

 
time_used1_warm = end - start
time_used1_warm = np.round(time_used1_warm, 3)

print("time_used1_warm: ", time_used1_warm)            


time_used1_warm:  0.186


In [12]:
start = time.time()
for i in range(N):
    for j in range(N):
#         if i != j:
        if i == 2:
            path1 = dijkstra_show_path_minimax_path(X_distance_matrix, i, j)
end = time.time()

 
time_used2_cold = end - start
time_used2_cold = np.round(time_used2_cold, 3)

print("time_used2_cold: ", time_used2_cold) 


time_used2_cold:  0.162


In [13]:
print("Ratio: ", np.round(time_used1_warm/time_used2_cold, 3))


Ratio:  1.148


In [14]:
# It seems a warm-start calculation of minimax path can be less efficient than cold-start. 
# The speculative reason is that the size of candidate_node_list is quite large.

